Imports

In [1]:
import joblib
import numpy as np
import tensorflow as tf
from collections import Counter

import sys
import os
import random
import math
import random
import multiprocessing as mp
import gc
import time
import datetime

from tensorflow.keras import datasets, layers, models

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

from etl_data_reader.etl_data_reader import ETL_data_reader
from etl_data_reader.etl_data_names import ETL_data_names
from etl_data_reader.etl_character_groups import ETL_character_groups

from sklearn.preprocessing import LabelBinarizer

# Change matplotlib to use a Japanese font
# to actually be able to display correctly
matplotlib.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'MS Gothic'
    }
)

Global variables, used throughout the program

In [2]:
debug = False
num_chars = 759  #759 per part.pkl
epochs = 150

#Set the path for input data and create the reader
path = "./ETL_kanji"
reader = ETL_data_reader(path)

In [3]:
# Debug method to see if images are loading correctly
def show_image(img : np.array, label : str):
    plt.figure(figsize=(2.2, 2.2))
    plt.title(label=label)
    plt.axis("off")
    plt.imshow(img.astype(np.float64), cmap="gray")

In [4]:
if debug:
    sample_img, sample_lable = reader.read_dataset_file(1, ETL_data_names.ETL10, ETL_character_groups.hiragana)
    show_image(sample_img[200], sample_lable[200])
    # free again
    del(sample_img)
    del(sample_lable)

## Load the data
Grab all the data from ETL9B (ETL10) and shuffle it

In [5]:
x, y =  reader.read_dataset_part(ETL_data_names.ETL10, ETL_character_groups.hiragana)    # load only the relevant dataset
#x, y = reader.read_dataset_whole([ETL_character_groups.kanji], 16)
print(x.shape, y.shape)

# because the data is ordered shuffle it
rng_state = np.random.get_state()
np.random.shuffle(x)
np.random.set_state(rng_state)
np.random.shuffle(y)

Loading all data set files (ETL10_x) from: ./ETL_kanji\ETL10...
(14271, 64, 64, 1) (14271,)


One-hot-encode the labels for the CNN, save the labels for later use

In [6]:
#one hot encode the labels
lb = LabelBinarizer()
lb.fit(y)
o_y = lb.transform(y)

# free the memory from the original string labels
del(y)

# concat classes-list to string
classes = [i for i in lb.classes_]
s = "".join([i for i in lb.classes_])

# save a list of all labels into a .txt file
with open("./labels.txt", "w+", encoding="utf8") as f:
    f.write(s)
    
# a string formatted as a python list can be easily evaluated 
# therefore lets save this representation too
with open("./labels_python_list.txt", "w+", encoding="utf8") as f:
    f.write(str(classes))

In [7]:
# store the images in a shared array and free the loaded data
memory_size_x = x.shape[0] * x.shape[1] * x.shape[2] * x.shape[3]
x_shared = mp.RawArray("f", memory_size_x // 2)
x_np = np.frombuffer(x_shared, dtype="float16").reshape(x.shape)
np.copyto(x_np, x)
del(x)

# store the labels in a shared array and free the loaded data
memory_size_y = o_y.shape[0] * o_y.shape[1]
y_shared = mp.RawArray('b', memory_size_y)
y_np = np.frombuffer(y_shared, dtype="b").reshape(o_y.shape)
np.copyto(y_np, o_y)
del(o_y)

gc.collect()

show_image(x_np[200], lb.inverse_transform(np.array([y_np[200]])))

In [8]:
import DataGenerator

train_generator = DataGenerator.DataGenerator(len(x_np), 2048, 
                                            80, "training", 
                                            x_shared, y_shared, 
                                            x_np.shape, y_np.shape,
                                            8, True)
test_generator = DataGenerator.DataGenerator(len(y_np), 2048, 
                                            20, "testing", 
                                            x_shared, y_shared, 
                                            x_np.shape, y_np.shape,
                                            8, True)

In [ ]:
train_x, train_y = train_generator.__getitem__(10)
test_x, test_y   = test_generator.__getitem__(10)

show_image(train_x[23], lb.inverse_transform(np.array([train_y[23]])))

Load data, print some for verification

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(input_shape=(64, 64, 1), kernel_size=3, activation='relu', filters=32, name="conv2D_1_2_input"),
        tf.keras.layers.Conv2D(kernel_size=3, activation='relu', filters=32, name="conv2D_1_1"),
        tf.keras.layers.Conv2D(kernel_size=3, activation='relu', filters=32, name="conv2D_1_2"),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, name="maxpool_1"),

        tf.keras.layers.Conv2D(kernel_size=3, activation='relu', filters=32, name="conv2D_2_1"),
        tf.keras.layers.Conv2D(kernel_size=3, activation='relu', filters=32, name="conv2D_2_2"),

        tf.keras.layers.Conv2D(kernel_size=3, activation='relu', filters=64, name="conv2D_3_1"),
        tf.keras.layers.Conv2D(kernel_size=3, activation='relu', filters=64, name="conv2D_3_2"),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, name="maxpool_3"),

        tf.keras.layers.Conv2D(kernel_size=3, activation='relu', filters=128, name="conv2D_4_1"),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, name="maxpool_4"),

        tf.keras.layers.Flatten(name="flatten_1"),
        tf.keras.layers.Dropout(0.25, name="dropout_1"),
        
        tf.keras.layers.Dense(2048, name="dense_1"),
        tf.keras.layers.Dropout(0.1, name="dropout_2"),
        tf.keras.layers.Dense(2048, name="densfasave_1"),
        tf.keras.layers.Dropout(0.1, name="dropourstat_2"),

        tf.keras.layers.Dense(2048, name="dense_2"),
        tf.keras.layers.Dropout(0.25, name="dropout_3"),
        tf.keras.layers.Dense(2048, name="densearst_2"),
        tf.keras.layers.Dropout(0.25, name="dropouart_3"),

        tf.keras.layers.Dense(len(lb.classes_),  name="dense_3"),

        #set the dtype to float32 for numerical stability
        tf.keras.layers.Softmax(dtype="float32", name="softmax_1_output") 
    ], name="model")

if debug:
    print(model.output_shape)
    model.summary()

In [ ]:
#path where the model should be saved
model_dir = os.path.join(os.path.dirname(os.getcwd()), "model")
print(model_dir)

Train the model

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001,
                                beta_1=0.9,
                                beta_2=0.999,
                                epsilon=1e-08,)


model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    x=train_generator,
    epochs=150,
    initial_epoch=0,
    validation_data=test_generator,
    max_queue_size=400,
    callbacks=callbacks_list,
    workers=1)

Display model information

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1.0])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print(test_acc)

In [ ]:
debug = False

if debug:
    y_pred = np.argmax(model.predict(X_test), axis=1)
    y_true = y_test
    
    #y_pred = le.inverse_transform(y_pred)
    #y_true = le.inverse_transform(y_test)

    labels = np.unique(le.inverse_transform(y_test))

    conf_mtx = tf.math.confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(100, 80))
    sns.heatmap(conf_mtx, xticklabels=labels, yticklabels=labels, 
                annot=True, fmt='g')
    plt.xlabel('Prediction')
    plt.ylabel('Label')
    plt.show()
